<a href="https://colab.research.google.com/github/peronvidal/tp-grupal-3-marias/blob/master/An%C3%A1lisis_modelo_arbol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
linear_model = LinearRegression()
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import sys
import seaborn as sns
from sklearn.metrics import confusion_matrix
# sns.set(rc={'figure.figsize':(15,5)})

In [ ]:
!wget https://github.com/peronvidal/tp-grupal-3-marias/raw/master/datasets/intervenciones-de-seguridad-vial/intervenciones-de-seguridad-clean.xlsx

--2023-06-11 21:30:49--  https://github.com/peronvidal/tp-grupal-3-marias/raw/master/datasets/intervenciones-de-seguridad-vial/intervenciones-de-seguridad-clean.xlsx
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/peronvidal/tp-grupal-3-marias/master/datasets/intervenciones-de-seguridad-vial/intervenciones-de-seguridad-clean.xlsx [following]
--2023-06-11 21:30:49--  https://raw.githubusercontent.com/peronvidal/tp-grupal-3-marias/master/datasets/intervenciones-de-seguridad-vial/intervenciones-de-seguridad-clean.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 495977 (484K) [application/octet-

In [ ]:
df = pd.read_excel('intervenciones-de-seguridad-clean.xlsx')

In [ ]:

## Aca se agregan columnas con datos relacionados a fechas
df['fecha_original'] = df['fecha']
df['fecha'] = pd.to_datetime(df['fecha']) # todas las fechas a datetime de pandas
df['year'] = pd.DatetimeIndex(df['fecha']).year
df['month'] = pd.DatetimeIndex(df['fecha']).month
df['day'] = pd.DatetimeIndex(df['fecha']).month
# Define a function to apply the condition
def condition(row):
    if row['lesionados'] > 0 or row['fallecidos']>0:
        return 1
    else:
        return 0

# Apply the function to create a new column
# df['NewColumn'] = df.apply(lambda row: condition(row), axis=1)

df['heridos_o_fallecidos'] = df.apply(lambda row: condition(row), axis=1)
df.loc[df['moto'] > 0, 'moto'] = 1
df.loc[df['liviano'] > 0, 'liviano'] = 1
df.loc[df['bus'] > 0, 'bus'] = 1
df.loc[df['camion'] > 0, 'camion'] = 1

In [ ]:
from sklearn.model_selection import train_test_split
X = pd.get_dummies(data=df, columns=['condiciones_meteorologicas','autopista','superficie_de_la_via','moto','liviano','camion','bus'])
X = pd.concat([X,df[['month','day']]],axis=1)
y = df['heridos_o_fallecidos']
X = X.drop(["heridos_o_fallecidos","fecha","pk","banda_y_o_ramal","lesionados","fallecidos","year","fecha","tipo_de_siniestro","hora","fecha_original"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42)
# bolsa de sueños: una forma mas facil de conseguir solo los dummies

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
import numpy as np

# Define the parameter grid
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None] + list(range(1, 21)),
    'min_samples_split': [2, 5, 10, 15, 20],
    'min_samples_leaf': [1, 2, 4, 8, 16]
}

# Create a decision tree classifier object
clf = DecisionTreeClassifier()

# Create a KFold object for cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=kfold)

# Fit the GridSearchCV object to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best Parameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)


Best Parameters:  {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 1, 'min_samples_split': 2}
Best Score:  0.7618348200440648


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

clf = DecisionTreeClassifier(criterion='gini',max_depth=1, min_samples_leaf=1, min_samples_split=2)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Exactitud:", accuracy)

from sklearn.metrics import confusion_matrix

confusion_mat = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:")
print(confusion_mat)

from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred)
print("Precisión:", precision)
recall = recall_score(y_test, y_pred)
print("Exhaustividad:", recall)
f1 = f1_score(y_test, y_pred)
print("Valor-F:", f1)



Exactitud: 0.7598971722365039
Matriz de confusión:
[[965 141]
 [326 513]]
Precisión: 0.7844036697247706
Exhaustividad: 0.6114421930870083
Valor-F: 0.6872069658405894


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score


# Crear el pipeline con los pasos de preprocesamiento y el modelo
pipeline = Pipeline([
    ('preprocessing', StandardScaler()),  # Paso de preprocesamiento: escalamiento de características
    ('model', DecisionTreeClassifier(criterion='gini',max_depth=1, min_samples_leaf=1, min_samples_split=2))  # Paso del modelo: Regresión Logística
])

# Realizar la validación cruzada con el pipeline
accuracy = cross_val_score(pipeline, X, y, cv=10, scoring='accuracy')
recall =cross_val_score(pipeline, X, y, cv=10, scoring='recall')
# Imprimir los resultados de la validación cruzada
print("Scores de validación cruzada:", accuracy)
print("Promedio del score accuracy:", accuracy.mean())
print("Scores de validación cruzada:", recall)
print("Promedio del score recall:", recall.mean())

Scores de validación cruzada: [0.74935733 0.77506427 0.7622108  0.70051414 0.75192802 0.7403599
 0.77763496 0.81209781 0.79279279 0.75160875]
Promedio del score accuracy: 0.7613568765239715
Scores de validación cruzada: [0.55855856 0.64156627 0.63253012 0.54216867 0.68072289 0.67771084
 0.7560241  0.70783133 0.60843373 0.55421687]
Promedio del score recall: 0.6359763377835665
